# Part 1: Extract and Transform Event Data

**import packages**

In [1]:
from data_store import DataStore
from extract import Extract
import pandas

Use the Extract class extract and tranform event_data files

* Find all datafile
* Combine into a single dataframe with pandas
* Drop nas on the 'artist' column
* Export to CSV

In [2]:
extract = Extract('./event_data')
columns = ['artist','firstName','gender','itemInSession','lastName','length', 'level','location','sessionId','song','userId']
dropna  = ['artist']
extract.save('event_datafile_new.csv', columns, dropna)

Let's take a peek at the data to make sure it looks right

In [3]:
data_frame = pandas.read_csv('event_datafile_new.csv')
data_frame.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Stephen Lynch,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",829,Jim Henson's Dead,91
1,Manowar,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Shell Shock,73
2,Morcheeba,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Women Lose Weight (Feat: Slick Rick),73
3,Maroon 5,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Won't Go Home Without You,73
4,Train,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Hey_ Soul Sister,73


# Part 2: Load and Query the Extracted Data

### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

Start by defining the Session class to model user listening sessions

The primary keys for this model are: 

```sql
    PRIMARY KEY (sessionId, itemInSession)
```

These keys ensure that:

* we can query first by `sessionId` and `itemInSession`.
* `sessionId` is unqiue to a user.
* `itemInSession` when combine with session ensures uniqueness within a session.

In [4]:
class Session(DataStore):
    primary_keys = ['sessionId', 'itemInSession']
    select_keys  = ['artist', 'song', 'length']
    columns      = {
        'sessionId':     'int',
        'itemInSession': 'int',
        'artist':        'text',
        'song':          'text',
        'length':        'double'
    }

Now use the model to load and query event data

In [6]:
session = Session()
session.load('./event_datafile_new.csv')

In [13]:
session.where('sessionid = 338 AND itemInSession = 4')

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Start by defining the User Session class to model artists listened to in a session

The primary keys for this model are: 

```sql
    PRIMARY KEY (userId, sessionId, itemInSession)
```

These keys ensure that:

* we can query first by `userId`, and then `sessionId`.
* `userId` provides for the unique user when quering for user session listens.
* `itemInSession` is added to providing sorting to the resultant output.

In [15]:
class UserSession(DataStore):
    primary_keys = ['userId', 'sessionId', 'itemInSession']
    select_keys  = ['artist', 'song', 'firstName', 'lastName']
    columns      = {
        'userId':        'int',
        'sessionId':     'int',
        'itemInSession': 'int',
        'artist':        'text',
        'song':          'text',
        'firstName':     'text',
        'lastName':      'text',
    }

In [16]:
user_session = UserSession()
user_session.load('./event_datafile_new.csv')
user_session.where('userId = 10 AND sessionId = 182')

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Start by defining the User Listens class to model songs listened to by users

The primary keys for this model are: 

```sql
    PRIMARY KEY (song, userId)
```

These keys ensure that:

* we can query by `song`.
* `userId` provides for the uniquness of the user when quering for listens.

In [17]:
class UserListen(DataStore):
    primary_keys = ['song', 'userId']
    select_keys  = ['firstName', 'lastName']
    columns      = {
        'song':      'text',
        'userId':    'int',
        'artist':    'text',
        'firstName': 'text',
        'lastName':  'text'
    }

In [18]:
user_listen = UserListen()
user_listen.load('./event_datafile_new.csv')
user_listen.where("song = 'All Hands Against His Own'")

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


## Drop the tables before closing out the sessions

In [10]:
session.drop_table()
user_session.drop_table()
user_listen.drop_table()

## Close the session and cluster connection

In [11]:
session.shutdown()
user_session.shutdown()
user_listen.shutdown()